In [1]:
import io.github.cdimascio.dotenv.Dotenv
import java.nio.file.Paths
import com.fasterxml.jackson.databind.JsonNode
import org.litote.kmongo.*

In [2]:
import org.bson.BsonDocument

val client = KMongo.createClient("mongodb://localhost:42692/")
val db = client.getDatabase("s5_snyk_libio")

In [3]:
val patchCommitCollection = db.getCollection("patchCommitsLibio")

In [4]:
import com.mongodb.client.*

private class MongoCursorIterable<T>(private val cursor: MongoCursor<T>) : MongoCursor<T> by cursor, Iterable<T> {

    override fun iterator(): Iterator<T> = cursor
}

private fun <T> MongoIterable<T>.kCursor(): MongoCursorIterable<T> = MongoCursorIterable(iterator())

fun <T, R> MongoIterable<T>.useCursor(block: (Iterable<T>) -> R): R {
    return kCursor().use(block)
}

In [5]:
import com.github.javaparser.JavaParser

fun getClassNameOfJavaSource(javaSrc: String): String?
{
    val cu = JavaParser().parse(javaSrc)
    val packageName = cu.result.get().packageDeclaration.get().name.toString()

    val typeDec = cu.result.get().types.firstOrNull()

    val res = if (typeDec == null) null
    else (if (packageName.isNotEmpty()) "$packageName." else "") + typeDec.name.toString()

    return res
}

In [6]:
patchCommitCollection.countDocuments()

1153

In [7]:
val patchCommitRecordsWithModifiedClass = patchCommitCollection.find().useCursor { blk ->
    blk.map {
        it["modifiedClass"] = try {
            getClassNameOfJavaSource(it["modifiedFileSrcBefore"] as String)
        } catch (e: Exception) {
            null
        }
        it
    }.filterNotNull()
}.toList().also { println(it.count()) }

1153


In [8]:
patchCommitRecordsWithModifiedClass
    .forEach {
        patchCommitCollection.replaceOneById(it["_id"]!!, it)
    }